In [1]:
import pandas as pd

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/home/codespace/anaconda3/envs/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("data/aa_dataset-tickets-multi-lang-5-2-50-version.csv")

df_eng = df[df.language == "en"]
df_eng.head()

,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51,Account,Disruption,Outage,IT,Tech Support,NaN,NaN,NaN
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51,Product,Feature,Tech Support,NaN,NaN,NaN,NaN,NaN
3,Inquiry Regarding Invoice Details,"Dear Customer Support Team,\n\nI hope this mes...",We appreciate you reaching out with your billi...,Request,Billing and Payments,low,en,51,Billing,Payment,Account,Documentation,Feedback,NaN,NaN,NaN
4,Question About Marketing Agency Software Compa...,"Dear Support Team,\n\nI hope this message reac...",Thank you for your inquiry. Our product suppor...,Problem,Sales and Pre-Sales,medium,en,51,Product,Feature,Feedback,Tech Support,NaN,NaN,NaN,NaN
5,Feature Query,"Dear Customer Support,\n\nI hope this message ...",Thank you for your inquiry. Please specify whi...,Request,Technical Support,high,en,51,Feature,Product,Documentation,Feedback,NaN,NaN,NaN,NaN


In [7]:
df_eng["body"].isnull().sum()

np.int64(0)

In [13]:
num = 0
for i in df_eng.body:
    if "<" in i:
        print(i)

# print(num)

Dear Customer Support,<br><br>I am submitting a report concerning a critical problem within the billing system of the financial organization. The issue involves errors that are impacting several products related to investment data analytics and optimization. It appears that the problem is probably due to integration issues within the system. Our team has attempted to review invoices and reset payment gateways, but unfortunately, we have not yet found a solution.<br><br>The errors are causing incorrect billing amounts and payments, leading to considerable disruptions in business operations. We are worried that this problem may continue to affect our activities.
Dear Support Team,<br>I am reaching out to inquire about healthcare data security solutions and hospital infrastructure services. Could you kindly provide details about the types of services you offer and the benefits they provide to organizations? I would appreciate it if you could include information on data encryption techniqu

In [5]:
df_eng['combined'] = df_eng['subject'] + ' [SEP] ' + df_eng['body']
df_eng['combined'].head()

/tmp/ipykernel_4843/806511140.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eng['combined'] = df_eng['subject'] + ' [SEP] ' + df_eng['body']


1    Account Disruption [SEP] Dear Customer Support...
2    Query About Smart Home System Integration Feat...
3    Inquiry Regarding Invoice Details [SEP] Dear C...
4    Question About Marketing Agency Software Compa...
5    Feature Query [SEP] Dear Customer Support,\n\n...
Name: combined, dtype: object

In [15]:
df_eng["combined"][1]

'Account Disruption [SEP] Dear Customer Support Team,\\n\\nI am writing to report a significant problem with the centralized account management portal, which currently appears to be offline. This outage is blocking access to account settings, leading to substantial inconvenience. I have attempted to log in multiple times using different browsers and devices, but the issue persists.\\n\\nCould you please provide an update on the outage status and an estimated time for resolution? Also, are there any alternative ways to access and manage my account during this downtime?'

In [6]:
len(df_eng)

16338

In [6]:
# Load model directly
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# A function to translate text

def translate(text, target_lang):
    inputs = tokenizer(text, return_tensors="pt")
    translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids(target_lang))
    return tokenizer.batch_decode(translated_tokens, skip_special_tokens = True)[0]

    
# Convert to Yoruba, Igbo and Hausa
df_eng["yoruba"] = df_eng["combined"].apply(lambda x: translate(x, "yor_Latn"))

"""
text= "Account Disruption [SEP] Dear Customer Support Team,\\n\\nI am writing to report a significant problem with the centralized account management portal, which currently appears to be offline. This outage is blocking access to account settings, leading to substantial inconvenience. I have attempted to log in multiple times using different browsers and devices, but the issue persists.\\n\\nCould you please provide an update on the outage status and an estimated time for resolution? Also, are there any alternative ways to access and manage my account during this downtime?"
model_inputs = tokenizer(text, return_tensors = "pt")

# Translate to Yoruba
gen_tokens = model.generate(**model_inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids("yor_Latn"))
print(tokenizer.batch_decode(gen_tokens, skip_special_tokens = True)[0])
"""

KeyboardInterrupt: 